# **Import Modules & Libraries**
* ignite is a high-level library to help with training and evaluating neural networks in PyTorch flexibly and transparently. It supports Weights & Biases handler to log metrics, model/optimizer parameters, gradients during training and validation. 

In [1]:
!pip install pytorch-ignite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 34.8 MB/s 


In [32]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor
from ignite.handlers import EarlyStopping 
from ignite.engine import Engine, Events # Runs a given process_function over each batch of a dataset, emitting events as it goes.
from ignite.utils import setup_logger # Ignite makes use of handlers to configure what we want to log / suivre le logging
from ignite.contrib.handlers import TensorboardLogger

# **Model Definition**

In [33]:
class MLP(nn.Module):
  def __init__(self, input_dim, N1, N2, output_dim):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(input_dim,N1)
    self.fc2 = nn.Linear(N1, N2)
    self.fc3 = nn.Linear(N2, output_dim)
    self.dropout = nn.Dropout(0.2)
    self.relu = nn.ReLU()
  
  def forward(self, x):
    out=self.dropout(self.relu(self.fc1(x)))
    out=self.dropout(self.relu(self.fc2(out)))
    out=self.fc3(out)        
    return out

# **Data Preparation**

In [34]:
def prepare_data(batch_size):
  # transform to normalize the data
  transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
  
  # Download and load the training and test datasets
  trainset = MNIST(root='MNIST', download=True, train=True, transform=transform)
  testset = MNIST(root='MNIST', download=True, train=False, transform=transform)

  trainset_size = round(len(trainset) * 0.8)
  validset_size = len(trainset) - trainset_size
  trainset, validset = random_split(trainset, [trainset_size, validset_size])

  train_dl = DataLoader(trainset, batch_size=batch_size, shuffle=True)
  valid_dl = DataLoader(validset, batch_size=batch_size, shuffle=True)
  test_dl = DataLoader(testset, batch_size=batch_size, shuffle=False)

  return train_dl, valid_dl, test_dl

# **Setting**

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP(784, 128, 64, 10)
model.to(device)  # Move model before creating optimizer
print(model)
optimizer = Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()

MLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
)


# **Training**

In [36]:
def train_step(engine, batch):
  model.train()
  optimizer.zero_grad()
  x, y = batch[0].to(device), batch[1].to(device)
  y_pred = model(x.view(x.shape[0], -1))
  loss = criterion(y_pred, y)
  loss.backward()
  optimizer.step()
  return loss.item()

def validation_step(engine, batch):
  model.eval()
  with torch.no_grad():
    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    outputs = model(inputs.view(inputs.shape[0], -1))
    _,predicted = torch.max(outputs.data, 1)      
    accuracy = (predicted == labels).sum().item()/labels.size(0)
  return accuracy

In [41]:
def run(model, device, optimizer, criterion, epochs, train_dl, valid_dl):

  # Trainer
  trainer = Engine(train_step)
  trainer.logger = setup_logger("trainer")

  # Evaluator
  Evaluator = Engine(validation_step)
  Evaluator.logger = setup_logger("Evaluator")
  from ignite.engine import create_supervised_evaluator
# Accuracy and loss metrics are defined
  val_metrics = {
  "accuracy": Accuracy(),
  "loss": Loss(criterion)
}
  
  def score_function(engine):
    return Evaluator.state.output

  # Early stopping handler
  handler = EarlyStopping(patience=2, score_function=score_function, trainer=trainer)
  Evaluator.add_event_handler(Events.COMPLETED, handler)
  # Define a Tensorboard logger
  tb_logger = TensorboardLogger(log_dir="quick-start-mnist-output")

# Attach handler to plot trainer's loss every 100 iterations
  tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batchloss": loss},
)


  
  @trainer.on(Events.ITERATION_COMPLETED(every=100))
  def log_training_loss(engine):
    print(f"ITERATION - Loss: {engine.state.output:.2f}")
        
  @trainer.on(Events.EPOCH_COMPLETED)
  def log_training_results(engine):
    Evaluator.run(train_dl)
    print(f"Training Results - Epoch: {trainer.state.epoch} Accuracy: {Evaluator.state.output:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED)
  def log_validation_results(engine):
    Evaluator.run(valid_dl)
    print(f"Validation Results - Epoch: {trainer.state.epoch} Accuracy: {Evaluator.state.output:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED | Events.COMPLETED)
  def log_time(engine):
    print(f"{trainer.last_event_name.name} took { trainer.state.times[trainer.last_event_name.name]} seconds")

  trainer.run(train_dl, max_epochs=epochs)
  tb_logger.close()

In [42]:
if __name__ == "__main__":
  # Data Preparation
  train_dl, valid_dl, test_dl = prepare_data(batch_size=64)

  run(model, device, optimizer, criterion, 10, train_dl, valid_dl)

2022-11-22 11:14:09,927 trainer INFO: Engine run starting with max_epochs=10.


ITERATION - Loss: 0.14
ITERATION - Loss: 0.33
ITERATION - Loss: 0.14
ITERATION - Loss: 0.31
ITERATION - Loss: 0.32
ITERATION - Loss: 0.19
ITERATION - Loss: 0.23


2022-11-22 11:14:18,709 Evaluator INFO: Engine run starting with max_epochs=1.
2022-11-22 11:14:26,517 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:07.806
2022-11-22 11:14:26,518 Evaluator INFO: Engine run complete. Time taken: 00:00:07.808
2022-11-22 11:14:26,524 Evaluator INFO: Engine run starting with max_epochs=1.


Training Results - Epoch: 1 Accuracy: 0.97


2022-11-22 11:14:28,455 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01.929
2022-11-22 11:14:28,456 Evaluator INFO: Engine run complete. Time taken: 00:00:01.931
2022-11-22 11:14:28,461 trainer INFO: Epoch[1] Complete. Time taken: 00:00:18.531


...

ITERATION - Loss: 0.15


2022-11-22 11:14:45,851 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:08.236
2022-11-22 11:14:45,853 Evaluator INFO: Engine run complete. Time taken: 00:00:08.238
2022-11-22 11:14:45,858 Evaluator INFO: Engine run starting with max_epochs=1.


Training Results - Epoch: 2 Accuracy: 0.97


2022-11-22 11:14:47,749 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01.888
2022-11-22 11:14:47,751 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
2022-11-22 11:14:47,758 trainer INFO: Terminate signaled. Engine will stop after current iteration is finished.
2022-11-22 11:14:47,760 Evaluator INFO: Engine run complete. Time taken: 00:00:01.899
2022-11-22 11:14:47,762 trainer INFO: Engine run complete. Time taken: 00:00:37.833


Validation Results - Epoch: 2 Accuracy: 0.97
EPOCH_COMPLETED took 9.144144058227539 seconds
COMPLETED took 37.833301067352295 seconds


**TensorBoardLogger** handler allows to log metric results, model’s and optimizer’s parameters, gradients, and more during the training and validation for TensorBoard.

In [43]:
%load_ext tensorboard

%tensorboard --logdir=.

<IPython.core.display.Javascript object>